<a href="https://colab.research.google.com/github/charlesfrye/data-structures/blob/main/HillClimber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook randomly generates strings to match a target -- in this case, a short Shakespeare sentence.

The fully random algorithm runs in exponential time,
while the hill climber runs much faster (linear?).

In [ ]:
!pip install wandb

In [ ]:
!wandb login

In [ ]:
import math
import random
import string

In [ ]:
import wandb

In [ ]:
TARGET = "methinks it is like a weasel"
letters = string.ascii_lowercase + " "

def iterate(candidate, target=TARGET):
  new_candidate = update(candidate, target)
  return new_candidate

def score(candidate, target=TARGET):
  return sum(match(candidate, target))

def match(candidate, target=TARGET):
  return [char_s == char_t for char_s, char_t in zip(candidate, target)]

def smart_update(candidate, target=TARGET):
  is_matchings = match(candidate, target)
  rand_chars = random_guess(target)
  return "".join(char if is_matching else random_char() for
                 char, is_matching in zip(candidate, is_matchings))

def random_guess(candidate, target=TARGET):
  return "".join(random.choices(letters, k=len(target)))

def random_char():
  return random.choice(letters)

In [ ]:
for seed in range(100):

  config = {"method": "hill-climb",
            "target": TARGET,
            "max_iters": 1e5,
            "seed": seed}

  with wandb.init(project="monkeys", config=config, mode="online") as run:
    random.seed(wandb.config.seed)
    if wandb.config.method == "hill-climb":
      update = smart_update
    elif wandb.config.method == "random":
      update = random_guess

    tbl = wandb.Table(
      columns=["score", "string", "update_index", "counter"])
    
    candidate = random_guess("", TARGET)
    counter, update_index = 0, -1
    best_score = -math.inf
    best_candidate = candidate

    while counter <= wandb.config.max_iters:
      _score = score(candidate)
      if _score > best_score:
        best_score = _score
        best_candidate = candidate
        update_index += 1
        run.log({
            "best_score": best_score,
            "update_index": update_index,
            "counter": counter})
        print(best_candidate, best_score)
        tbl.add_data(best_score, candidate, update_index, counter)
      if _score == len(TARGET):
        break
      
      candidate = iterate(candidate)
      counter += 1
    run.log({"progress": tbl})